# Taller Grupal 1 - Métodos y aplicaciones de analítica I

**Anderson Alvarado & Juan Felipe Gonzalez Rodriguez**

# Librerias

In [14]:
# instalar aprioricon pip
%pip install efficient-apriori
%pip install mlxtend

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from efficient_apriori import apriori
from mlxtend.frequent_patterns import apriori as mlx_apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

In [2]:
%run "Funciones/funciones.ipynb"

{HUEVO} -> {CREMA} (conf: 1.000, supp: 0.667, lift: 1.500, conv: 333333333.333)
{CREMA} -> {HUEVO} (conf: 1.000, supp: 0.667, lift: 1.500, conv: 333333333.333)


In [3]:
# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("coolwarm")
plt.rcParams["figure.figsize"] = (12, 8)

# Carga de Datos

In [4]:
# Cargar order_products_train.csv
order_products_train = pd.read_csv('Data/order_products__train.csv')
order_products_train.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


In [5]:
# cargar: Data/aisles.csv
aisles = pd.read_csv('Data/aisles.csv')
aisles.head(10)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


In [6]:
# cargar: Data/departments.csv
departments = pd.read_csv('Data/departments.csv')
departments.head(10)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [7]:
# cargar: Data/products.csv
products = pd.read_csv('Data/products.csv')
products.head(10)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


In [9]:
# unir los dataframes
order_products_train = order_products_train.merge(products, on='product_id')
order_products_train = order_products_train.merge(aisles, on='aisle_id')
order_products_train = order_products_train.merge(departments, on='department_id')
order_products_train.head(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
5,1,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,1,47209,7,0,Organic Hass Avocado,24,4,fresh fruits,produce
7,1,22035,8,1,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
8,36,39612,1,0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs
9,36,19660,2,1,Spring Water,115,7,water seltzer sparkling water,beverages


In [10]:
order_products_train.shape

(1384617, 9)

# Descripción del dataset

In [11]:
# descripcion y analisis de datos
order_products_train.describe().T

,count,mean,std,min,25%,50%,75%,max
order_id,1384617.0,1.706298e+06,989732.648868,1.0,843370.0,1701880.0,2568023.0,3421070.0
product_id,1384617.0,2.555624e+04,14121.272449,1.0,13380.0,25298.0,37940.0,49688.0
add_to_cart_order,1384617.0,8.758044e+00,7.423936,1.0,3.0,7.0,12.0,80.0
reordered,1384617.0,5.985944e-01,0.490183,0.0,0.0,1.0,1.0,1.0
aisle_id,1384617.0,7.130423e+01,38.104094,1.0,31.0,83.0,107.0,134.0
department_id,1384617.0,9.839777e+00,6.292830,1.0,4.0,8.0,16.0,21.0


In [12]:
order_products_train.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
product_name         0
aisle_id             0
department_id        0
aisle                0
department           0
dtype: int64

# Análisis Dataset

# Preguntas

## a) Están interesados en saber qué productos o categorías de productos se venden bien juntos.

## b) A partir de ello, desean recomendaciones específicas separadas para su tienda virtual y sus tiendas físicas.

## c) Finalmente, están curiosos acerca de qué otra información podría darles las diversas bases de datos allí presentes que pudiese ser valiosa para comprender las dinámicas del comportamiento de sus usuarios /productos.